# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

In [10]:
import util as ut
import ImgViewer as iv
import cv2
import numpy as np
# ImgUtil provides all utility functions for image manipulation
import ImgUtil as iu
# image view provides a mechanism(mostly for debug)to push images as a pipeline is being developed and 
# see then 3 x 3 or more at a time 
import ImgViewer as iv
# LaneUtils implements the core of our object model and various "lane focused" things
# Lanes:  contain LaneBoundary(s) and attributes common to both boundaries
# LaneBoundary(s): contain attributes for a particular lane bounday
# Windows: are used to calculate lane boundariesimport LaneUtils as lu
import LaneUtils as lu

In [12]:
#
# run with the viewer enabled almost everywhere to see the output of each stage of the image pipeline
#

#cd is cache_dict, pd is parm_dict
cd, pd = ut.app_init(viewer=True, saver=True, title="whatever")
vwr = cd['viewer']

def doit():
    lane = lu.Lane(cd, pd, vwr=vwr)

    for path in ut.get_fnames("test_images/", "*.jpg", max=1): # stop at 1 iter to gather pipe stages 4 writepup
        in_img = iu.imRead(path, reader='cv2', vwr=vwr)
        out_img= lane.lane_finder_pipe(in_img, cd, pd, vwr=vwr)
    print("check the test_out directory")

doit()

findChessboardCorners(camera_cal/calibration5.jpg) failed
findChessboardCorners(camera_cal/calibration1.jpg) failed
findChessboardCorners(camera_cal/calibration4.jpg) failed
check the test_out directory
